In [1]:
import csv
import os
from visigoth import Diagram
from visigoth.containers import Map, Box
from visigoth.map_layers import WMTS, Hexbin
from visigoth.common import Text, MapLayerManager, Legend
from visigoth.utils.colour import ContinuousPalette
from visigoth.utils.mapping import Projections

import requests

In [2]:
# caution - this will be a 1.8 Gb download
filename = "yellow_tripdata_2015-01.csv"
if not os.path.exists(filename):
    print("Downloading data, please wait")
    r = requests.get('https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-01.csv', stream=True)
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)

In [11]:
bounds = ((-74.2, 40.6), (-73.5, 40.9))

In [13]:
data = []
headers = {}

# load the data from CSV, for January 1st 2015 only 
rdr = csv.reader(open("yellow_tripdata_2015-01.csv","r"))

for row in rdr:
    if not headers:
        for idx in range(len(row)):
            headers[row[idx]] = idx
    else:
        dt = row[headers["tpep_pickup_datetime"]]
        if dt.startswith("2015-01-01"):
            for (lat_field,lon_field) in [("pickup_latitude","pickup_longitude"),("dropoff_latitude","dropoff_longitude")]:
                lat = float(row[headers[lat_field]])
                lon = float(row[headers[lon_field]])
                data.append((lon,lat))

In [14]:
d = Diagram(fill="white",margin_right=0,margin_left=0,margin_top=0,margin_bottom=0)

# create the map and and an open street map base layer
m = Map(width=768,boundaries=bounds,projection=Projections.EPSG_3857,zoom_to=16)
wmts = WMTS(embed_images=False)
m.add(wmts)

palette = ContinuousPalette(colourMap=["#00FF0080","#0000FFC0","#FF0000FF"])

# define the heatmap
heatmap = Hexbin(data,palette=palette,nr_bins_across=150,stroke_width=0.3,stroke="#DDDDDD40",min_freq=1)
m.add(heatmap)

# lay out the diagram, starting with a title
d.add(Text("NY Yellow Cab pickup/dropoff locations - Jan 1st 2015",font_height=32))
d.add(Text("Total Trips: %d"%(int(len(data)/2))))

# add the legend and map
d.add(Box(m))
d.add(Legend(palette,label="Frequency (thousands)",labelfn=lambda x:str(int(x/1000))))

# add a layer manager to allow layers to be turned on and off
mlm = MapLayerManager([{"layer":heatmap,"label":"Trips"},{"layer":wmts,"label":"Base Map"}],title="Select Layer(s)",height=150)
d.add(mlm)
d.connect(mlm,"manage_layers",m,"manage_layers")

In [15]:
from IPython.display import IFrame,display

html = d.draw(format="html",include_footer=False)
with open("ny.html","w") as f:
    f.write(html)
display(IFrame("./ny.html",850,850))

hexbin building 99%   #################################################
hexbin complete
